# Calculating 5-sigma Limiting Mag for LSST
___

## Rationale

This value is quoted in the opsim output, but it is being questioned. Understanding the derivation (and how to extract the image ZP) is important 

In [ ]:
%matplotlib inline
# %matplotlib notebook

from __future__ import print_function

try:
    from importlib import reload
except:
    pass

import os
import sys
import warnings

import numpy as np
import pandas as pd
import astropy as ap
import sfdmap

import sqlite3
from sqlalchemy import create_engine

from astropy.constants import h
from scipy.integrate import simps

from matplotlib import pyplot as plt
import matplotlib.colors as mpl_colors 

import astropy.units as u

In [ ]:
import pyCoCo as pccsims
import pycoco as pcc

In [ ]:
# %%timeit ## 1 loop, best of 3: 1min 31s per loop 
opsimdbpath = os.environ.get('OPSIMDBPATH')
print(opsimdbpath)
# opsimdbpath = "/Users/berto/data/LSST/OpSimOutputDBs/astro_lsst_01_1068_sqlite.db"
# opsimdbpath = "/Users/berto/data/LSST/OpSimOutputDBs/minion_1016_sqlite.db"

conn = create_engine('sqlite:///'+opsimdbpath, echo = False)
# opsimdf = pd.read_sql_table('Summary', con=conn)

In [ ]:
opsimdf = pd.read_sql_query(sql = "SELECT * FROM Summary WHERE night > 365 AND night < 731", con = conn)

opsimdf.columns

In [ ]:
opsimdf.columns

# OpSim m5 derivation 

Recreating the m5 values, as per http://smtn-002.lsst.io/en/latest/

\begin{split}m5 = C_m + dC_m + 0.50\,(m_{sky} - 21.0) + 2.5 log_{10}(0.7 /
FWHM_{eff}) \\
+ 1.25 log_{10}(expTime / 30.0) - k_{atm}\,(X-1.0)\end{split}

\begin{split}dC_m = dC_m^{inf} - 1.25 log_{10}(1 + (10^{(0.8\, dC_m^{inf} -
1)}/Tscale)\end{split}

\begin{split}Tscale = expTime / 30.0 * 10.0^{-0.4*(m_{sky} - m_{darksky})}\end{split}


## Tscale

In [ ]:
opsimdf["filtSkyBrightness"].hist()

In [ ]:
def calc_Tscale(df):


    
    

    Tscale = df["visitExpTime"].head()/30. * np.power(10., -0.4*(df["filtSkyBrightness"] - mdarksky))
    return Tscale

In [ ]:
dark_sky_path = os.path.join(os.environ["LSST_THROUGHPUTS_BASELINE"], "darksky.dat")
darksky = pcc.SpectrumClass()
darksky.load(dark_sky_path, wavelength_u=u.nm, flux_u=u.cgs.erg / u.si.cm ** 2 / u.si.s / u.nm,
             fmt="ascii.commented_header", wmin=3500 * u.angstrom, wmax=11000 * u.angstrom, )

lsst_filters = [i for i in opsimdf["filter"].unique()]

zp_dict = {}
m_sky_dict = {}

for filter_key in lsst_filters:
    zp_dict[filter_key] = pcc.kcorr.calc_AB_zp("LSST_"+filter_key)
    zp_dict["LSST_"+filter_key] = zp_dict[filter_key]
    
    m_sky[filter_key] = -2.5 * np.log10(pcc.kcorr.calc_spectrum_filter_flux(filter_key, darksky)) - zp_dict[filter_key]
zp_dict   

In [ ]:
opsimdf["zp"] = opsimdf["filter"].map(zp_dict)

In [ ]:
def calc_dCm():
    dCm = False
    return dCm


def calc_m5():
    m5 = False
    return m5